## Building a min GPT with Transformer

> Reference : [Andrej Karpathy Notebook](https://colab.research.google.com/drive/1JMLa53HDuA-i7ZBmqV7ZnA3c_fvtXnx-?usp=sharing#scrollTo=h5hjCcLDr2WC)

### Constants

In [1]:
import torch


device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(8745)
batch_size = 32 # batch size
block_size = 16 # context length (number of tokens in the input)
emmbed_size = 32 # embedding size of the Q, K, V
num_heads = 4 # number of heads in the multi-head attention
num_layers = 4 # number of transformer blocks

### Download and Load the Data

In [2]:
# the tiny shakespeare dataset
# check if the file exists
import os
if not os.path.exists("input.txt"):
    print("Downloading the dataset...")
    !wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt 

# read the file and display some lines
data = ""
with open('input.txt', 'r') as f:
    data = f.read()

print(f"len of data = {len(data)}\n")
print(data[:100])


len of data = 1115394

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


#### Create the vocabulary from the data (Tokenize)

In [3]:
# create vocabulary
vocab = sorted(list(set(data)))

print("".join(vocab))
vocab_size = len(vocab)
print(f"{vocab_size=}")

# Tokenize
stoi = {char:i for i, char in enumerate(vocab)}
itos = {i:s for s,i in stoi.items()}

encode = lambda s: [stoi[c] for c in s]
decode = lambda s: "".join([itos[c] for c in s]) # you just return a string

print(encode("Hello there"))
print(decode(encode("Hello there")))


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
vocab_size=65
[20, 43, 50, 50, 53, 1, 58, 46, 43, 56, 43]
Hello there


#### Create the Train, Test split and Batching

In [4]:

data_t = torch.tensor(encode(data), dtype=torch.long)
data_t = data_t.to(device)

# train test split
n = int(0.9*len(data_t))
train_data = data_t[:n]
val_data = data_t[n:]

print(f"{len(train_data)=}, {len(val_data)=}")


def get_batch(split):
    data = train_data if split == 'train' else val_data
    random_indices = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in random_indices])
    y = torch.stack([data[i+1:i+block_size+1] for i in random_indices])
    return x, y

sample_X, sample_Y = get_batch("train")
print("X", sample_X.shape)
print("Y", sample_Y.shape)

len(train_data)=1003854, len(val_data)=111540
X torch.Size([32, 16])
Y torch.Size([32, 16])


### Model Creation

In [5]:
from torch import nn
import torch.nn.functional as F
from torchinfo import summary


@torch.no_grad()
def estimate_loss(model, eval_iters):
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    def __init__(self, emmbed_size, head_size, block_size):
        super().__init__()
        self.Q = nn.Linear(emmbed_size, head_size, bias=False) # query
        self.K = nn.Linear(emmbed_size, head_size, bias=False) # key
        self.V = nn.Linear(emmbed_size, head_size, bias=False) # value
        # mask to prevent the model from attending to future tokens
        self.register_buffer("tril", torch.tril(torch.ones(block_size, block_size)))
        # scaling factor
        self.dsqrt = head_size ** 0.5

    def forward(self, x):
        # x: B, T, Emb (batch size, context length, embedding size)
        B, T, Emb = x.shape
        Q = self.Q(x)  # B, T, Head
        K = self.K(x)  # B, T, Head
        V = self.V(x)  # B, T, Head
        # attention scores
        att = (Q @ K.transpose(-2, -1)) / self.dsqrt  # B, T, T
        # mask the attention scores for future tokens
        att = att.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        # softmax to get the attention weights
        att = F.softmax(att, dim=-1)
        # weighted sum of the values
        out = att @ V  # B, T, Head
        return out

class MultiHeadAttention(nn.Module):
    def __init__(self, emmbed_size, num_heads, block_size):
        super().__init__()
        head_size = emmbed_size // num_heads
        # each head has its own Q, K, V
        self.heads = nn.ModuleList(
            [Head(emmbed_size, head_size, block_size) for _ in range(num_heads)]
        )

    def forward(self, x):
        # x: B, T, Emb (batch size, context length, embedding size)
        return torch.cat([head(x) for head in self.heads], dim=-1)

class FeedForward(nn.Module):
    def __init__(self, emmbed_size):
        super().__init__()
        # hidden size is 4 times the embedding size 
        hidden_size = 4 * emmbed_size
        # feedforward network
        self.net = nn.Sequential(
            nn.Linear(emmbed_size, hidden_size),
            nn.GELU(),
            nn.Linear(hidden_size, emmbed_size),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    def __init__(self, emmbed_size, num_heads, block_size):
        super().__init__()
        # multi-head attention layer (series of self-attention heads)
        self.sa = MultiHeadAttention(emmbed_size, num_heads, block_size)
        self.ff = FeedForward(emmbed_size)
        self.ln1 = nn.LayerNorm(emmbed_size)
        self.ln2 = nn.LayerNorm(emmbed_size)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ff(self.ln2(x))
        return x

class SimpleGPT(nn.Module):
    def __init__(self, vocab_size, emmbed_size, block_size, num_heads, num_layers):
        super().__init__()
        # each token directly maps to a vector
        self.token_embedding_table = nn.Embedding(vocab_size, emmbed_size)
        # each position directly maps to a vector
        self.position_embedding_table = nn.Embedding(block_size, emmbed_size)
        # transformer blocks
        self.blocks = nn.Sequential(
            *[Block(emmbed_size, num_heads, block_size) for _ in range(num_layers)]
        )
        # final layer norm
        self.ln_f = nn.LayerNorm(emmbed_size)
        # final linear layer to map the embedding to the vocabulary size
        self.lm_head = nn.Linear(emmbed_size, vocab_size)

    def forward(self, idx, targets=None):
        # B: batch size, T: context length
        B, T = idx.shape
        # token embeddings
        tok_embs = self.token_embedding_table(idx)                          # B, T, Emb
        # position embeddings
        pos_embs = self.position_embedding_table(torch.arange(T, device=idx.device))  # T, Emb
        # add the token and position embeddings
        x = tok_embs + pos_embs                                             # B, T, Emb
        x = self.blocks(x)                                                  # B, T, Emb
        x = self.ln_f(x)                                                    # B, T, Emb
        logits = self.lm_head(x)                                            # B, T, V

        # if targets is None, we are generating
        if targets is None:
            loss = None
        else:
            # reshape the logits and targets to be 2D
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, idx, max_new_tokens):
        # set the model to evaluation mode
        self.eval()
        for _ in range(max_new_tokens):
            # get the context length (last block_size tokens)
            idx_cond = idx[:, -self.position_embedding_table.num_embeddings:]
            # get the logits and loss
            logits, _ = self(idx_cond)
            # get the last token
            logits = logits[:, -1, :]
            # get the probabilities
            probs = F.softmax(logits, dim=-1)
            # sample from the probabilities
            idx_next = torch.multinomial(probs, num_samples=1)
            # concatenate the new token to the context
            idx = torch.cat((idx, idx_next), dim=1)
        return idx



model = SimpleGPT(vocab_size, emmbed_size, block_size, num_heads, num_layers)
model.to(device)
summary(model, input_data=sample_X , depth=6, row_settings=["var_names"])

Layer (type (var_name))                       Output Shape              Param #
SimpleGPT (SimpleGPT)                         [32, 16, 65]              --
├─Embedding (token_embedding_table)           [32, 16, 32]              2,080
├─Embedding (position_embedding_table)        [16, 32]                  512
├─Sequential (blocks)                         [32, 16, 32]              --
│    └─Block (0)                              [32, 16, 32]              --
│    │    └─LayerNorm (ln1)                   [32, 16, 32]              64
│    │    └─MultiHeadAttention (sa)           [32, 16, 32]              --
│    │    │    └─ModuleList (heads)           --                        --
│    │    │    │    └─Head (0)                [32, 16, 8]               --
│    │    │    │    │    └─Linear (Q)         [32, 16, 8]               256
│    │    │    │    │    └─Linear (K)         [32, 16, 8]               256
│    │    │    │    │    └─Linear (V)         [32, 16, 8]               256
│    │    │  

#### Model Sample Output without Training

In [6]:
start = torch.zeros((1, 1), dtype=torch.long, device=device)
out = model.generate(start, 1000)
print(decode(out[0].tolist()))


VMPnQ-fW3UqC'khQws:iP;pjCs$cQpDj!:vy,lDWNp,AdFQUrxVf z'R.;UbPBmiOppoASwL$PJa:'dmvwmuuCes:EHrL,P!N,WJ
JNdJNqooV.oaqioGowa-;-Ii:FYoJ:Jo&'wql?NZF?rQRrDjSuhQG$m $k'$XPIyQNOm,ueKskSMhKYobRZDlbZ!M:'wmN&::CpqrqGw3LciaoxTToOgAp,g$PXSwHzT::'E
B!-sXxcq-g'sVAQ:$lwhmMyq.WkhCHlcWQ:qPvedoy,QNSlgQ:-nCoO:pfqYztoHoywrKY':WMlt&$Y:UCg!MHhew.oQrbDmhOwDCDmPAdbIGs.kN$3j$PhrQY:M.:BQggwVFzu,ml,,rtJ:pGRNFl.HDI'Axw-rrgvB
rjCqhQhfww3-,wM?iwQUM3!oQuua.zFhO:jXwQrd$:a'AgrcCyWTlk-.t dQ;NApZNRrv'Qs-ady.NFpC&d wSrMoHaV,UgnMQ,.nzw$wj'DhAh&v.Nz&lK.ylSlf;IvzYQT3:-Zb?.C3qB:$YmVAGmXQKX-kM&bRr.e'XkTt:Zzp,YpqSeLu,p;Y:$Jzk,-cobgXwmAKQa:Fwi,GyJhSRyFl.M WqAz'J
lf,hEjmoyVyb?PsDKo;Sl;lQ-ahjT$$khCO$oRDC.'WhQHqp bJ?p!sye-myasHgUbb'EX$$H.A fYD::NFk,squrpAJyv,X3Nb..NO ,:qroCYhZf$'Jz-,ThmAhQ,tis3!Rhuahnj FaSsmXrPblB,CAZ'3bqTycFt&bY$jr;h'uN!prJn!urI?P.isHfYr'TokpU,G!-rmGO$$o?.:rQP-LGi;NVwqsO
EquFFkXxqDSqSyJOOgcBuceAjY'&YG-w,W' J:o,uqr;;.YmlkvDmDfdpEYGyu,anlIBH-XlrbAfztwqGIMMj$qngleRk$,rGDJ
lqxOSksy.$yny,f&Lsmq.mN Wsqnl!3'wQTuAp;PDj-Ni

### Train the Model

In [7]:
# Train the model

optimzer = torch.optim.AdamW(model.parameters(), lr=1e-3)

max_iters = 5001
eval_interval = 1000
eval_iters = 100

for iter in range(max_iters):
    if iter % eval_interval == 0:
        losses = estimate_loss(model, eval_iters)
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    X, Y = get_batch('train')
    logits, loss = model(X, Y)
    optimzer.zero_grad(set_to_none=True)
    loss.backward()
    optimzer.step()


step 0: train loss 4.3159, val loss 4.3150
step 1000: train loss 2.2040, val loss 2.2360
step 2000: train loss 2.0423, val loss 2.1043
step 3000: train loss 1.9271, val loss 2.0357
step 4000: train loss 1.8843, val loss 1.9922
step 5000: train loss 1.8446, val loss 1.9871


#### Model Sample Output after Training

In [8]:
start = torch.zeros((1, 1), dtype=torch.long, device=device)
out = model.generate(start, 1000)
print(decode(out[0].tolist()))


That would them kingh jour all nevil prayingbither
This meo.


IRINCEO:
My those uft all stramentsing. I'll be seeps.

HOFlaranding
That:
Theus manainst if in endener's rue tonge?
In liveing allrage,
Of that yet in shig,
Whath se the for a fall tup good I hath faress
To Off Shis parmin:
If arwn thee times faith be a,
But sendlam,
shard dear!
Pacest tam,
And dost tarmys lether,
sir. I would Edwarm ardom, doth merus that thou o deasterss foges, that prain,
Not the res his lidatad. I with thy sech redoth
Than hath
She fighters'd hide a canless; as butich of thus dedsprer's thergs the good her
Chill lame:
Plaris of degren
This sinrelt;
Shamb, no shalls, who jud they loved's not I lot warrest, and for deds, your secrowbby men'd prard.

Shis time, triving?

BROLAN3,
I say 'Tis trigness of these endland.

JULINCER:S this sethings,
And burss, done,
Anight what yeawart if queen. That mall name tell the him disfiness to shomistelfispors your seep
So green eard oange did;- may to liges
As lend,
